In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
train = train.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)
test = test.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)

tar = train.loc[:,['start_latitude','start_longitude']].values
s_min_lat = min(train['end_latitude'].min(),train['start_latitude'].min())
s_min_long = min(train['end_longitude'].min(),train['start_longitude'].min())
e_max_lat = max(train['start_latitude'].max(),train['end_latitude'].max())
e_max_long = max(train['start_longitude'].max(),train['end_longitude'].max())

restrict_lat = [(e_max_lat-s_min_lat)*((i)/3) for i in range(4)]
restrict_long = [(e_max_long-s_min_long)*((i)/6) for i in range(7)]
restrict_lat[0] = -1
restrict_long[0] = -1
target = []
cnt = 0
for i in range(len(restrict_lat)-1):
    for j in range(len(restrict_long)-1):
        for x in tar:
            if (s_min_lat+restrict_lat[i] < x[0] <= s_min_lat+restrict_lat[i+1]) and (s_min_long+restrict_long[j] < x[1] <= s_min_long+restrict_long[j+1]):
                target.append(cnt)
        cnt += 1
train['restrict'] = target

tar = test.loc[:,['start_latitude','start_longitude']].values
# s_min_lat = min(train['end_latitude'].min(),train['start_latitude'].min())
# s_min_long = min(train['end_longitude'].min(),train['start_longitude'].min())
# e_max_lat = max(train['start_latitude'].max(),train['end_latitude'].max())
# e_max_long = max(train['start_longitude'].max(),train['end_longitude'].max())

# restrict_lat = [(e_max_lat-s_min_lat)*((i)/3) for i in range(4)]
# restrict_long = [(e_max_long-s_min_long)*((i)/6) for i in range(7)]
# restrict_lat[0] = -1
# restrict_long[0] = -1
target = []
cnt = 0
for i in range(len(restrict_lat)-1):
    for j in range(len(restrict_long)-1):
        for x in tar:
            if (s_min_lat+restrict_lat[i] < x[0] <= s_min_lat+restrict_lat[i+1]) and (s_min_long+restrict_long[j] < x[1] <= s_min_long+restrict_long[j+1]):
                target.append(cnt)
        cnt += 1
test['restrict'] = target

In [2]:
#root = '/Users/junho/Desktop/git_repo/pycharmProjects/pythonProject/dacon/jeju'

from haversine import haversine, Unit, haversine_vector
start = [(i-90,j) for i,j in zip(train['start_longitude'],train['start_latitude'])]
end = [(i-90,j) for i,j in zip(train['end_longitude'],train['end_latitude'])]
distance = [haversine(i, j, unit=Unit.KILOMETERS) for i,j in zip(start,end)]

# train = train.drop(['start_longitude','end_longitude','start_latitude','end_latitude'],axis=1)
train['distance'] = distance

from haversine import haversine, Unit, haversine_vector
start = [(i-90,j) for i,j in zip(test['start_longitude'],test['start_latitude'])]
end = [(i-90,j) for i,j in zip(test['end_longitude'],test['end_latitude'])]
distance = [haversine(i, j, unit=Unit.KILOMETERS) for i,j in zip(start,end)]

# train = train.drop(['start_longitude','end_longitude','start_latitude','end_latitude'],axis=1)
test['distance'] = distance
#train = train[train['connect_code'] == 0]
#train = train[train['multi_linked'] == 0]
# train = train[train['maximum_speed_limit'] != 40]

# import glob
# rain_path = glob.glob('./rain_temp/rn*.csv')
# tar = [pd.read_csv(i,header=6,encoding='cp949') for i in rain_path]
# ans = pd.DataFrame()
# for i in tar:
#     ans = pd.concat([ans,i],axis=0)
# ans = ans.groupby('날짜').mean()
# ans = ans.fillna(0)
# ans = ans.reset_index()
# ans['날짜'] = ans['날짜'].str.replace('-','')
# ans['날짜'] = pd.to_numeric(ans['날짜'])
# ans.columns = ['base_date','loc','rain']
# ans = ans.drop('loc',axis=1)
# train = pd.merge(ans,train)
# test = pd.merge(ans,test)

# temperature_path = glob.glob('./rain_temp/ta*.csv')
# tar = [pd.read_csv(i,header=6,encoding='cp949') for i in temperature_path]
# ans = pd.DataFrame()
# for i in tar:
#     ans = pd.concat([ans,i],axis=0)
# ans = ans.groupby('날짜').mean()
# ans = ans.reset_index()
# ans['날짜'] = ans['날짜'].str.replace('-','')
# ans['날짜'] = ans['날짜'].str.replace('\t','')
# ans['날짜'] = pd.to_numeric(ans['날짜'])

# ans.columns = ['base_date','loc','temp','low','high']
# ans = ans.drop(['loc','low','high'],axis=1)
# train = pd.merge(ans,train)
# test = pd.merge(ans,test)

# train['base_date'] = train['base_date'].astype('object')
# train['base_date'] = [str(i)[4:6] for i in train['base_date']]
# test['base_date'] = [str(i)[4:6] for i in test['base_date']]

from sklearn.preprocessing import LabelEncoder
str_col = ["base_hour","day_of_week","multi_linked","connect_code",'road_rating'
,"weight_restricted","road_type","start_longitude",'start_turn_restricted','end_turn_restricted'
,"road_name","start_node_name","end_node_name",'end_longitude','start_latitude','end_latitude'
,'restrict'] # "maximum_speed_limit"
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])

    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

from sklearn.decomposition import PCA
pca = PCA(n_components=1)
target = ['start_turn_restricted','end_turn_restricted',
          'start_latitude','end_latitude','lane_count']
x1 = pca.fit_transform(train.loc[:,target])
x2 = pca.fit_transform(test.loc[:,target])

train['x'] = x1
test['x'] = x2

train = train.drop(target,axis=1)
test = test.drop(target,axis=1)

train = train.sort_values(by='id')
test = test.sort_values(by='id')

In [3]:
train.head()

,id,base_date,day_of_week,base_hour,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,start_node_name,start_longitude,end_node_name,end_longitude,target,restrict,distance,x
0,TRAIN_0000000,06,1,17,106,46,0,0,60.0,1,1,386,464,386,462,52.0,0,0.030806,53.669074
1,TRAIN_0000001,07,1,21,103,34,0,0,60.0,0,0,32,322,2,321,30.0,0,0.484324,342.167663
2,TRAIN_0000002,10,4,7,103,36,0,0,80.0,0,0,426,102,179,100,61.0,0,0.722014,-198.767604
3,TRAIN_0000003,03,0,13,107,57,0,0,50.0,0,0,58,388,203,388,20.0,0,0.117894,-392.513879
4,TRAIN_0000004,10,6,8,103,35,0,0,80.0,0,0,267,68,269,72,38.0,0,0.402515,151.957233


In [16]:
sorted(list(train['restrict'].value_counts().index))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [17]:
sorted(list(test['restrict'].value_counts().index))

[0, 1, 2, 3, 5, 8, 9, 10, 11, 12, 13, 14]

In [9]:
test['restrict'].value_counts()

13    92089
2     79114
3     50339
1     30569
14    26368
9      4394
8      3860
12     2942
10      755
0       739
5        36
11       36
Name: restrict, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=10
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.1, random_state=10
)

In [18]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.linear_model import LogisticRegression

def rf_speed(speed, train, test):
    #     train = pd.read_parquet('./train.parquet')
    #     test = pd.read_parquet('./test.parquet')
    #     train = train.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)
    #     test = test.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)

#     train = train.drop(train[(train['distance'] > 0.8) & (train['target'] < 5)].index)
#     train = train.drop(train[train['target'] > 120].index)

    train = train[train["restrict"] == speed]
    y_train = train['target']
    X_train = train.drop(['id', 'target'], axis=1)

    test = test[test["restrict"] == speed]
    test_id = test['id']
    test = test.drop(['id'], axis=1)

    prediction_table = pd.DataFrame(test_id)

    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.08, random_state=10
    )

    r1 = RandomForestRegressor(random_state=10, bootstrap=200, oob_score=True)
#     r2 = CatBoostRegressor(random_state=10, iterations=5000, verbose=False)
#     r3 = ExtraTreesRegressor(random_state=10)
#     r4 = xgboost.XGBRegressor(random_state=10)
#     vote = VotingRegressor([('r1', r1), ('r2', r2), ('r3', r3), ('r4',r4)])

#     parms = {'verbose': [False]}
#     grid_cv = GridSearchCV(vote, param_grid=parms, cv=8, n_jobs=-1,
#                            scoring='neg_mean_absolute_error')  # ,param_grid=parms
    r1.fit(X_train, y_train)
#     r = LogisticRegression(random_state=10)
#     r.fit(X_train, y_train)
    
    pred = r1.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    print(mae)

    prediction_table['target'] = r1.predict(test)
    prediction_table.to_csv(f'./target{speed}.csv')
    print(f'save {speed}')

In [21]:
rf_speed('abc',train,test)

3.2900476769593796
save abc


In [19]:
for i in sorted(list(test['restrict'].value_counts().index)):
    rf_speed(i,train,test)

ans = pd.concat([pd.read_csv('target30.csv'),pd.read_csv('target50.csv'),
          pd.read_csv('target60.csv'),pd.read_csv('target70.csv'),
          pd.read_csv('target80.csv')]).drop('Unnamed: 0',axis=1)
ans = ans.sort_values('id')
ans = ans.set_index('id')
ans.to_csv('./submit.csv')

3.6488282712202977
save 0
3.4496741389981516
save 1
3.4317601832275852
save 2
3.4594934997305833
save 3
3.881836011527551
save 5
3.7342696726838884
save 8
3.4868067065829
save 9
3.50825002521082
save 10
3.6025117692658806
save 11
3.4518126939226423
save 12
3.412113161682694
save 13
3.492283157675256
save 14


In [22]:
tp = pd.read_csv('./targetabc.csv')
tp.drop('Unnamed: 0',axis=1,inplace=True)
tp.set_index('id',inplace=True)
tp.to_csv('./targetabc.csv')

In [34]:
res1 = pd.read_csv('./target30.csv')
res2 = pd.read_csv('./targetabc.csv')

In [39]:
res = pd.merge(res1,res2,on='id').drop('Unnamed: 0',axis=1)
res['target'] = res.mean(axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_12052\203849471.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  res['target'] = res.mean(axis=1)


In [43]:
res.loc[:,['id','target']].set_index('id').to_csv('res.csv')

In [33]:
pd.read_csv('./target30.csv')

,Unnamed: 0,id,target
0,0,TEST_000000,14.704167
1,1,TEST_000001,24.090833
2,2,TEST_000002,30.943333
3,3,TEST_000003,26.953333
4,4,TEST_000004,13.332167
...,...,...,...
291236,291236,TEST_291236,34.920000
291237,291237,TEST_291237,20.797500
291238,291238,TEST_291238,22.000000
291239,291239,TEST_291239,41.426000
